In [8]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()

In [2]:
import torch
from torch import nn
import torch.utils.data

In [12]:
import numpy as np

## 実装
- クロスエントロピー
- 仮説

In [3]:
criterion = nn.CrossEntropyLoss() # 損失の定義

In [11]:
preds = torch.tensor([[0.2, 0.8]]) # 二値分類
labels = torch.tensor([1]) # 正解ラベル

loss = criterion(preds, labels)
print(loss)

tensor(0.4375)


## 演習
- 3×3のモデル
- クロスエントロピー

In [19]:
X = torch.tensor(np.arange(12).reshape(4, 3)).float()
Y = torch.tensor([1, 2, 0, 1])
model = nn.Linear(3, 3)

In [20]:
criterion(model(X), Y)

tensor(3.3651, grad_fn=<NllLossBackward>)

## Pytorchで
- 勾配降下
- ニュートン法

で ルート2を求める

- 勾配降下の場合
  - $f(x) = x^3 - 6x$で計算する

In [47]:
def F(x):
    return x ** 3 -  6 * x

In [72]:
x = torch.tensor([1.0], requires_grad=True)
y = F(x)

In [73]:
learning_rate = 0.1

In [74]:
for i in range(10):
    y = F(x)
    z = torch.autograd.grad(y, x)
    x = x - learning_rate * z[0]
    print(i, "loop", x)

0 loop tensor([1.3000], grad_fn=<SubBackward0>)
1 loop tensor([1.3930], grad_fn=<SubBackward0>)
2 loop tensor([1.4109], grad_fn=<SubBackward0>)
3 loop tensor([1.4137], grad_fn=<SubBackward0>)
4 loop tensor([1.4141], grad_fn=<SubBackward0>)
5 loop tensor([1.4142], grad_fn=<SubBackward0>)
6 loop tensor([1.4142], grad_fn=<SubBackward0>)
7 loop tensor([1.4142], grad_fn=<SubBackward0>)
8 loop tensor([1.4142], grad_fn=<SubBackward0>)
9 loop tensor([1.4142], grad_fn=<SubBackward0>)


- ニュートン法の場合
  - $f(x) = x^3 - 6x$で計算する

In [75]:
def G(x):
    return x ** 2 - 2

In [92]:
x = torch.tensor([1.0], requires_grad=True)
y = G(x)

In [93]:
y

tensor([-1.], grad_fn=<SubBackward0>)

In [94]:
for i in range(10):
    y = G(x)
    y.backward()
    # backwardで値を入れる時はdataに代入
    # xに代入すると,backwardの対象でなくなる
    x.data = x.data - y/ x.grad
    print(i, "lootp", x)
    x.grad.zero_()

0 lootp tensor([1.5000], requires_grad=True)
1 lootp tensor([1.4167], requires_grad=True)
2 lootp tensor([1.4142], requires_grad=True)
3 lootp tensor([1.4142], requires_grad=True)
4 lootp tensor([1.4142], requires_grad=True)
5 lootp tensor([1.4142], requires_grad=True)
6 lootp tensor([1.4142], requires_grad=True)
7 lootp tensor([1.4142], requires_grad=True)
8 lootp tensor([1.4142], requires_grad=True)
9 lootp tensor([1.4142], requires_grad=True)


## Softmax回帰の実装
- irisに対し,softmax回帰で実装する

In [5]:
def train_epoch(model, data_loader):
    model.eval()
    for batch_idx, (data, target) in enumerate(data_loader): # 入力と正解
         optimizer.zero_grad() # Weightの初期化
         output = model(data) # 仮説で値代入
         loss = criterion(output, target) # 損失
         loss.backward() # 微分の計算
         optimizer.step() # パラメータの更新
         if batch_idx % 10 == 0:
             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                 batch_idx, batch_idx * len(data), len(data_loader.dataset),
                 100. * batch_idx / len(data_loader), loss.item()))

def valid_epoch(model, data_loader):
    model.train()
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader): # 入力と正解
             optimizer.zero_grad() # Weightの初期化
             output = model(data) # 仮説で値代入
             output.dtype
             loss = criterion(output, target) # 損失
             # 本来は全体でロスを数えて荷重平均を取る,accuracyを計算する

             if batch_idx % 10 == 0:
                 print('Test Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                     batch_idx, batch_idx * len(data), len(data_loader.dataset),
                     100. * batch_idx / len(data_loader), loss.item()))

In [30]:
iris = load_iris()
X_train, X_valid, y_train, y_valid = train_test_split(iris.data, iris.target, test_size=0.2)

X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train)
X_valid = torch.tensor(X_valid).float()
y_valid = torch.tensor(y_valid)

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
valid_dataset = torch.utils.data.TensorDataset(X_valid, y_valid)

model = nn.Linear(4, 3)

batch_size  = 120 # ミニバッチのデータの数
max_epoch = 100 #
train_loader = torch.utils.data.DataLoader(train_dataset, 
                   batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(train_dataset, 
                   batch_size=batch_size, shuffle=True)

criterion = nn.CrossEntropyLoss() # 損失の定義
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) #(確率的)勾配降下法

In [31]:
for epoch in range(max_epoch):
    train_epoch(model, train_loader)
    valid_epoch(model, valid_loader)

Train Epoch: 0 [0/120 (0%)]	Loss: 2.696589
Test Epoch: 0 [0/120 (0%)]	Loss: 2.541242
Train Epoch: 0 [0/120 (0%)]	Loss: 2.541242
Test Epoch: 0 [0/120 (0%)]	Loss: 2.408026
Train Epoch: 0 [0/120 (0%)]	Loss: 2.408026
Test Epoch: 0 [0/120 (0%)]	Loss: 2.289122
Train Epoch: 0 [0/120 (0%)]	Loss: 2.289121
Test Epoch: 0 [0/120 (0%)]	Loss: 2.179420
Train Epoch: 0 [0/120 (0%)]	Loss: 2.179420
Test Epoch: 0 [0/120 (0%)]	Loss: 2.076118
Train Epoch: 0 [0/120 (0%)]	Loss: 2.076119
Test Epoch: 0 [0/120 (0%)]	Loss: 1.977985
Train Epoch: 0 [0/120 (0%)]	Loss: 1.977985
Test Epoch: 0 [0/120 (0%)]	Loss: 1.884750
Train Epoch: 0 [0/120 (0%)]	Loss: 1.884750
Test Epoch: 0 [0/120 (0%)]	Loss: 1.796715
Train Epoch: 0 [0/120 (0%)]	Loss: 1.796714
Test Epoch: 0 [0/120 (0%)]	Loss: 1.714491
Train Epoch: 0 [0/120 (0%)]	Loss: 1.714491
Test Epoch: 0 [0/120 (0%)]	Loss: 1.638829
Train Epoch: 0 [0/120 (0%)]	Loss: 1.638828
Test Epoch: 0 [0/120 (0%)]	Loss: 1.570448
Train Epoch: 0 [0/120 (0%)]	Loss: 1.570448
Test Epoch: 0 [0/120 (

In [13]:
y_valid

array([1, 2, 1, 2, 2, 2, 0, 1, 1, 1, 0, 0, 0, 0, 2, 1, 1, 1, 2, 1, 1, 0,
       1, 2, 2, 1, 2, 1, 2, 1])

In [17]:
X_valid.shape

(30, 4)

In [18]:
X_train.shape

torch.Size([120, 4])